In [1]:
import torch
import scipy
import numpy as np

In [2]:
# load /vol/bitbucket/dm2223/info-theory-experiments/data/ecog_data_raw/Motion.mat

file_path = '/vol/bitbucket/dm2223/info-theory-experiments/data/ecog_data_raw/Motion.mat'

# Load the MATLAB file
mat_data = scipy.io.loadmat(file_path)

motion_time = torch.from_numpy(np.asarray(mat_data['MotionTime'])).squeeze()[1:]

print(motion_time.size())


torch.Size([125373])


In [3]:
ecog_time_path = "/vol/bitbucket/dm2223/info-theory-experiments/data/ecog_data_raw/ECoG_time.mat"

# Load the MATLAB file with ECoGTime key
mat_data = scipy.io.loadmat(ecog_time_path)
ecog_time_1000 = torch.from_numpy(np.asarray(mat_data['ECoGTime'])).squeeze()

# downsmaple

ecog_time_300 = ecog_time_1000[::3]


In [4]:
print(ecog_time_1000[-10:])
print(ecog_time_300[-10:])

tensor([1045.8190, 1045.8200, 1045.8210, 1045.8220, 1045.8230, 1045.8240,
        1045.8250, 1045.8260, 1045.8270, 1045.8280], dtype=torch.float64)
tensor([1045.8010, 1045.8040, 1045.8070, 1045.8100, 1045.8130, 1045.8160,
        1045.8190, 1045.8220, 1045.8250, 1045.8280], dtype=torch.float64)


In [5]:
num_motion_timesteps = motion_time.size(0)

print(num_motion_timesteps)

indices = []
for t in motion_time:
    differences = torch.abs(ecog_time_300 - t)
    index = torch.argmin(differences)
    indices.append(index.item())


125373


In [6]:

print(len(motion_time))

print(len(indices))

print(len(ecog_time_300[indices]))

125373
125373
125373


In [7]:
from models import SkipConnectionSupervenientFeatureNetwork

seed = 1

torch.manual_seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = {
    "torch_seed": seed,
    "dataset_type": "ecog",
    "num_atoms": 64,
    "batch_size": 1000,
    "train_mode": True,
    "train_model_B": False,
    "adjust_Psi": False,
    "clip": 5,
    "feature_size": 3,
    "epochs": 70,
    "start_updating_f_after": 500,
    "update_f_every_N_steps": 5,
    "minimize_neg_terms_until": 9999999999,
    "downward_critics_config": {
        "hidden_sizes_v_critic": [512, 1024, 1024, 512],
        "hidden_sizes_xi_critic": [512, 512, 512],
        "critic_output_size": 32,
        "lr": 1e-3,
        "bias": True,
        "weight_decay": 0,
    },
    "decoupled_critic_config": {
        "hidden_sizes_encoder_1": [512, 512, 512],
        "hidden_sizes_encoder_2": [512, 512, 512],
        "critic_output_size": 32,
        "lr": 1e-3,
        "bias": True,
        "weight_decay": 0,
    },
    "feature_network_config": {
        "hidden_sizes": [256, 256, 256, 256, 256],
        "lr": 1e-4,
        "bias": True,
        "weight_decay": 1e-3,
    }
}

model_path = "/vol/bitbucket/dm2223/info-theory-experiments/models/ECOG_feature_network_devout-leaf-5.pth"


skip_model = SkipConnectionSupervenientFeatureNetwork(
    num_atoms=config['num_atoms'],
    feature_size=config['feature_size'],
    hidden_sizes=config['feature_network_config']['hidden_sizes'],
    include_bias=config['feature_network_config']['bias'],
).to(device)


skip_model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [8]:

from datasets import ECoGDataset

dataset = ECoGDataset(prepare_pairs=False)

In [9]:
file_path = '/vol/bitbucket/dm2223/info-theory-experiments/data/ecog_data_raw/Motion.mat'

# Load the MATLAB file
mat_data = scipy.io.loadmat(file_path)

print(mat_data['MotionData'][0][0].shape)
print(mat_data['MotionData'][1][0].shape)
print(mat_data['MotionData'][2][0].shape)
print(mat_data['MotionData'][3][0].shape)
print(mat_data['MotionData'][4][0].shape)
print(mat_data['MotionData'][5][0].shape)


(125374, 3)
(125374, 3)
(125374, 3)
(125374, 3)
(125374, 3)
(125374, 3)


In [10]:
# clear cache
import torch
torch.cuda.empty_cache()

In [11]:
device = 'cuda'

downsampled_dataset = dataset[indices].to(device).float()

left_wrist = torch.tensor(mat_data['MotionData'][2][0])[1:]



model_path = "/vol/bitbucket/dm2223/info-theory-experiments/models/ECOG_feature_network_devout-leaf-5.pth"


skip_model = SkipConnectionSupervenientFeatureNetwork(
    num_atoms=config['num_atoms'],
    feature_size=config['feature_size'],
    hidden_sizes=config['feature_network_config']['hidden_sizes'],
    include_bias=config['feature_network_config']['bias'],
).to(device)


skip_model.load_state_dict(torch.load(model_path))

untrained_skip_model = SkipConnectionSupervenientFeatureNetwork(
    num_atoms=config['num_atoms'],
    feature_size=config['feature_size'],
    hidden_sizes=config['feature_network_config']['hidden_sizes'],
    include_bias=config['feature_network_config']['bias'],
).to(device)

V = skip_model(downsampled_dataset)

V_untrained = untrained_skip_model(downsampled_dataset)



In [12]:
# make a dataset clas with V as the x and left_wrist as the y

import torch
from torch.utils.data import Dataset, DataLoader

class MotionDataset(Dataset):
    def __init__(self, x_data, y_data):
        """
        Args:
            x_data (torch.Tensor): The input features - V
            y_data (torch.Tensor): The target labels - left_wrist
        """
        self.x_data = x_data.detach()
        self.y_data = y_data.detach()

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y



In [13]:

motion_dataset = MotionDataset(V, left_wrist)

motion_dataset_untrained = MotionDataset(V_untrained, left_wrist)

dataloader = DataLoader(motion_dataset, batch_size=100, shuffle=True)

dataloader_untrained = DataLoader(motion_dataset_untrained, batch_size=100, shuffle=True)


class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(3, 1024)
        self.fc2 = torch.nn.Linear(1024, 1024)
        self.fc3 = torch.nn.Linear(1024, 1024)
        self.fc4 = torch.nn.Linear(1024, 2048)
        self.fc5 = torch.nn.Linear(2048, 2048)
        self.fc6 = torch.nn.Linear(2048, 3)
        
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

mlp = MLP().to(device)

optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-3)

criterion = torch.nn.MSELoss()


num_epochs = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
losses = []
for epoch in range(num_epochs):
    for batch in dataloader:
        x, y = batch
        x, y = x.to(device).float(), y.to(device).float()

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = mlp(x)
        
        # Compute loss
        loss = criterion(outputs, y)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        losses.append(loss.item())


import matplotlib.pyplot as plt

plt.plot(losses)
plt.yscale('log')
plt.show()    

/vol/bitbucket/dm2223/info-theory-experiments/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([100, 3])) that is different to the input size (torch.Size([100, 2048])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (2048) must match the size of tensor b (3) at non-singleton dimension 1

In [ ]:


# same for untrained model

mlp_untrained = MLP().to(device)

optimizer_untrained = torch.optim.Adam(mlp_untrained.parameters(), lr=1e-3)

criterion = torch.nn.MSELoss()


num_epochs = 1

losses_untrained = []

for epoch in range(num_epochs):
    for batch in dataloader_untrained:
        x, y = batch
        x, y = x.to(device).float(), y.to(device).float()

        # Zero the parameter gradients
        optimizer_untrained.zero_grad()

        # Forward pass
        outputs = mlp_untrained(x)
        
        # Compute loss
        loss = criterion(outputs, y)
        
        # Backward pass and optimize
        loss.backward()
        optimizer_untrained.step()

        losses_untrained.append(loss.item())


plt.plot(losses_untrained)
plt.yscale('log')
plt.show()


RuntimeError: The size of tensor a (2048) must match the size of tensor b (3) at non-singleton dimension 1